# Morphing water vapor from SAPHIR
### Erfan Jahangir and Brian Mapes March 2017

#### General strategy of this notebook

Like any programming job, we work backward from a result-shaped container, and forward from the most precious original data input whose value we are trying to optimize. These intentions make it natural to build the middle steps and grab auxiliary inputs, as necessary to achieve success. Initially we achieve success in the simplest way, then later add sophistication -- but only to the extent it improves signal and reduces noise, as seen in the initial success of the data products. 

In this case, the results-shaped container is a single 1/2-degree (for now) array of 

1. **WV** (water vapor)
1. **AT** (the 'analysis tendency' of the morphing operation, representing all the physical source-sink terms). 
1. **t_early** (the time of the earlier observation that made the above products)
1. **t_late** (the time of the later observation " " " )

In this case, the precious data is swaths of data, made up of pixels, of WV500 retrieved from SAPHIR channels by Helene Brignietz and colleagues. These input data include arrays of: 

* the WV **value** at each pixel
* the **latitude** " " " 
* the **longitude** " " " 
* the **time of observation** " " " 

----------

In [11]:
import numpy as np 

In [28]:
import h5py

f = h5py.File('2012_07_01_1.mat','r')
variables = f.items()

for var in variables:
    name = var[0]
    data = var[1]
    print("Name ", name)  # Name
    if type(data) is h5py.Dataset:
        # If DataSet pull the associated Data
        # If not a dataset, you may need to access the element sub-items
        value = data.value
        print("Value", value)  # NumPy Array / Value


OSError: Unable to open file (file signature not found)

In [21]:
import scipy.io
mat = scipy.io.loadmat('')
mat.loadmat

AttributeError: 'dict' object has no attribute 'loadmat'

In [20]:
data = mat.get('swarm')

### Step 1: Create the results-shaped containers  

In [2]:
DX = 0.5 # degrees
SOUTH = -30 # Central latitudes of southern grid  cells
NORTH = 30  # " " northern " " 
WEST = 0.25 # Central longitude of westernmost cell
EAST = 359.75 # " " easternmost " 

# Build 1D spatial coordinate arrays 
NLAT = int( (NORTH-SOUTH)/DX )
lat = np.linspace(SOUTH, NORTH, NLAT)

NLON = int( (EAST-WEST)/DX )
lon = np.linspace(WEST, EAST, NLON)


# Now build containers for the results we desire
# Which order? LON,LAT? or LAT,LON? 

WV = np.zeros( (NLON,NLAT) )
AT = np.zeros( (NLON,NLAT) )

In [3]:
whos

Variable   Type       Data/Info
-------------------------------
AT         ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
DX         float      0.5
EAST       float      359.75
NLAT       int        120
NLON       int        719
NORTH      int        30
SOUTH      int        -30
WEST       float      0.25
WV         ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
lat        ndarray    120: 120 elems, type `float64`, 960 bytes
lon        ndarray    719: 719 elems, type `float64`, 5752 bytes
np         module     <module 'numpy' from '//a<...>kages/numpy/__init__.py'>


### Step 2: The things we need to fill the containers

To fill the above arrays, we will use the *time-proximity-weighted average* for WV500:  

$ WV500 = ( WV_{before}*dt_{after} + WV_{before}*dt_{after} )/(dt_{before} + dt_{after}) $ 

and the simplest *estimate of the time derivative* using the before and after observations:

$ AT500 = ( WV_{after} - WV_{before})/(dt_{before} + dt_{after}) $ 

**Thus, we need $ WV_{before}, WV_{after}, dt_{before}, dt_{after}  $**

----------------

In [4]:
# Containers for the necessary quantities to get our final products

WV_before = np.zeros( (NLON,NLAT) )
WV_after  = np.zeros( (NLON,NLAT) )
dt_before = np.zeros( (NLON,NLAT) )
dt_after  = np.zeros( (NLON,NLAT) )

In [5]:
whos

Variable    Type       Data/Info
--------------------------------
AT          ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
DX          float      0.5
EAST        float      359.75
NLAT        int        120
NLON        int        719
NORTH       int        30
SOUTH       int        -30
WEST        float      0.25
WV          ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_after    ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_before   ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
dt_after    ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
dt_before   ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
lat         ndarray    120: 120 elems, type `float64`, 960 bytes
lon         ndarray    719: 719 elems, type `float64`, 5752 bytes
np          module     <module 'numpy' from '//a<...>kages/numpy/__init__.py'>


## Now, how to get $ WV_{before}, WV_{after}, dt_{before}, dt_{after} $ ? 

#### We need two _time stacks_ of product-shaped lat-lon arrays. 

Two, because we need the VALUE and the TIME of each observation we put into the stack. 

These stack is a 3D array, centered on the product time (dt=0). It doesn't matter how long in time  this stack extends, as long as it is long enough that **every pixel in space has a before and an after observation**. That is, the DTMAX just has to be at least as big as the longest time gap between observations. Also, the time step between the layers in the stack just has to be short enough that we aren't wasting observations by over-writing some locations with multiple observations. Since the orbit time is about 100 minutes, 1 hour stacks are safe. 

Initially, let's make a container full of zeros. 

In [6]:
DTMAX = 6    # hours. The size of the centered stack will then be 2*DTMAX+1. 
DTstack = 1  # hour

WV_stack = np.zeros( (NLON,NLAT, 2*DTMAX+1) )
tobs_stack = np.zeros( (NLON,NLAT, 2*DTMAX+1) )

In [7]:
whos

Variable     Type       Data/Info
---------------------------------
AT           ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
DTMAX        int        6
DTstack      int        1
DX           float      0.5
EAST         float      359.75
NLAT         int        120
NLON         int        719
NORTH        int        30
SOUTH        int        -30
WEST         float      0.25
WV           ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_after     ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_before    ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_stack     ndarray    719x120x13: 1121640 elems, type `float64`, 8973120 bytes (8.55743408203125 Mb)
dt_after     ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
dt_before    ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
lat          ndarray    120: 120 elems, type

# Now the real data work is clear.

1. **How do we interrogate the stack** to get $ WV_{before}, WV_{after}, dt_{before}, dt_{after} $? 
    * For each location (lat,lon), the stack is a time series of length 2\*DTMAX+1. 
    * Need a method to find nonzero values closest to dt=0 (the center of the series). 
    * Special case: when there is an observation in dt=0, that is WV -- but what is AT?
1. **How do we fill the stack initially**, from the set of observation pixels within DTMAX of the time of the product? 
    * Loop over all the pixels in the input data within +/- DTMAX of the product hour. 
        * Each pixel has these values: {WVobs, tobs, latobs, lonobs}.
    * Place the pixel in the dt=0 layer of WV_stack and tobs_stack, at location (latobs, lonobs)
    * Place the pixel in the dt=1 layer at (latobs +v\*DT_stack\*1, lonobs +u\*DT_stack\*1)
        * this requires obtaining wind data u,v for that location (reanalysis u500,v500)
    * etc. for other dt layers in the stack
1. **How do we repeat the process for the next product hour?** 
    * We could start the whole process over from np.zeros(). Inefficient but safe. 
    * Better from efficieny perspective is to 
        * np.roll(WV_stack), np.roll(tobs_stack) to recenter the stack on the next hour.
        * WV_stack[:,:,DTMAX\*2]=0  to clobber where the periodic roll() operation has shifted past data into the future
        * Now just fill that future stack time, WV_stack[:,:,DTMAX\*2], with real data
            * this involves querying the satellite data up to DTMAX hours ahead of that time, shifting it all backward in time
            * (latobs -v\*DT_stack\*DTMAX, latobs -u\*DT_stack\*DTMAX)
            * (latobs -v\*DT_stack\*(DTMAX-1), latobs -u\*DT_stack\*(DTMAX-1))
            * etc. 